In [ ]:
import os 
import json
import time
import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cbook as cbook
#from scipy.misc import imread
from matplotlib.pyplot import imread

from importlib import reload
%load_ext autoreload
%autoreload 2
import AnalyzeAnnotations as aa
import PlotAnnotations as pa

from tqdm import tqdm_notebook

pd.options.display.max_colwidth = 400
sns.set()

### <span style="color:red"> Load sloth jsons </span> into array of pandas dataframes

In [ ]:
# load all json files from datapath into array of pandas dataframes
# path = './data/20191031_MIxedSpecies_Photos_final_annotations/'
# path = './data/testdata/'
json_path = './data/MEX_Sloth/Survey/'
imgs_path = './data/MEX_Sloth/Survey_photos/'


print(f"{time.strftime('%d/%m/%Y, %H:%M:%S', time.localtime())}:      Path to be loaded: {json_path}")
df_stats, all_df, json_files = aa.loadAllJSONSFromPath(json_path)
number_imgs = len([file for file in os.listdir(imgs_path) if os.path.isfile(os.path.join(imgs_path, file))])
print('Number of json files loaded: ' + str(len(all_df)))
print(f"Number of files in image folder: {number_imgs}")

#### Calculate abundance of classes + split positions + average pos column + length column + direction (rad)

In [ ]:
# update df_stats with abundance of classes
df_stats = aa.getNumberOfClassesInDFs(df_stats, all_df, json_files)
# extract head and tail positions + calculate average of head and tail + calc length + calc polarization
df_allsplit = aa.splitPos(df_stats, all_df, json_files)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_stats)
print(f"\n{df_allsplit[0].head(5)}")

#### Stats of dataframe df_stats

In [ ]:
#df.describe()
df_stats.describe()

#### Neighbours

* Calculate distance matrix and find neighbors
 * finds neighbors using a distance matrix and filtering out neighbours over two times average length and four times average length
* Add number of class neighbors column
 * adds columns for each fish containing the number of neighbors of each class (for distance d2/av2 and d4/av4)
* Add same class neighbors percentage and average number of neighbors (per class and total) to stats 
* degree: mean number of fish in radius of 2 body lengths
* density: degree / number of fish in image
* nnd: euclidean distances to nearest neighbor

In [ ]:
df_allsplit, df_stats, data_numbers = aa.neighbor_calculations(df_allsplit, df_stats, json_files, imgs_path)
# 
# check sample
#

pd.set_option("display.max_rows", None, "display.max_columns", None)

print(df_allsplit[4].head(2))
df_stats.head(3)

### Add average length, direction and polarization to stats

In [ ]:
df_stats = aa.pop_stats_pol_dir_len(df_allsplit, df_stats)

df_stats.head(3)

#### Export to csv

In [ ]:
if not os.path.exists('./output/stats'):
            os.makedirs('./output/stats')

for i, df_split in enumerate(df_allsplit):
    number = df_split.number
    df_split.to_csv('./output/stats/' + str(number) + '.csv')

df_stats.to_csv('./output/summary.csv')

### Plot positions

In [ ]:
#test one image
index=4
#print(json_files)
number = df_allsplit[index].number
imgpath = pa.get_imgpath_for_number(number, imgs_path)
pa.plot_pos_cat_img(df_allsplit[index], imgpath, show=True, save=True)
pa.plot_pos_ori_cat(df_allsplit[index], show=True, save=True)
pa.plot_pos_ori_cat_img(df_allsplit[index], imgpath, show=True, save=True)

In [ ]:
# Plot positions of all read files
pa.plot(df_allsplit, json_files, imgs_path, SHOW=False, SAVE=True)

print(f"{time.strftime('%d/%m/%Y, %H:%M:%S', time.localtime())}")

## TODO

* refactor much
* convert .ipynb to .py
* create exe file from python script

In [ ]:
angles = [0.163618, -0.208563, -0.246237, 0.379256, 0.189199]
angles_deg = np.degrees(angles)

print(angles_deg)

msin = np.mean(np.sin(angles))
mcos = np.mean(np.cos(angles))
np.degrees(np.arctan2(msin,mcos))




In [ ]:
v = [[1, 0],[0.5, 1]]
v_normalized = preprocessing.normalize(v, norm='l1')
print(v_normalized)
v_mean_unit = np.mean(v_normalized, axis=0)
print(v_mean_unit)


ord = 2
print(np.linalg.norm(v_mean_unit, ord=ord))
print(np.square(np.linalg.norm(v_mean_unit, ord=ord)))


v = [[1, 0],[1, 0.5]]
v_normalized = preprocessing.normalize(v, norm='l1')
print(v_normalized)
v_mean_unit = np.mean(v_normalized, axis=0)
print(v_mean_unit)


ord = 2
print(np.linalg.norm(v_mean_unit, ord=ord))
print(np.square(np.linalg.norm(v_mean_unit, ord=ord)))



In [ ]:
# v = [[0,2], [10,0], [-10,0]]
v = [[1,0], [1,1]]
print(f"pos:\n{v}")
v_normalized = preprocessing.normalize(v, norm='max')
print(f"normalized_pos:\n{v_normalized}")
v_mean = np.mean(v_normalized, axis=0)
print(f"mean normalized vector: {v_mean.tolist()}")
pol = np.linalg.norm(v_mean, ord=np.inf)
# pol = np.linalg.norm(v_mean)

print(f"pol: {pol}")

In [ ]:
# v = [[0,2], [10,0], [-10,0]]
v = [[0,1], [1,1]]

print(f"pos:\n{v}")
# # print(f"magnitudes: {np.linalg.norm(v, axis=0)}")
# v_normalized = preprocessing.normalize(v, norm='l2')
# print(f"normalized_pos:\n{v_normalized}")
# v_mean = np.mean(v_normalized, axis=0)
# print(f"mean normalized vector: {v_mean.tolist()}")
# pol = np.linalg.norm(v_mean, ord=np.inf)
# # pol = np.linalg.norm(v_mean)

unit_vectors = preprocessing.normalize(v, norm='l2')
print(f"unit vectors:\n{unit_vectors}")

mean_of_sum_of_unitv = np.mean(unit_vectors, axis=0)
print(f"mean unit vector: {mean_of_sum_of_unitv.tolist()}")

mag_meanv = np.linalg.norm(mean_of_sum_of_unitv)

pol = mag_meanv

print(f"pol: {pol}")

In [ ]:
# v1 = np.random.rand(10000,2)
# v2 = np.random.rand(10000,2)

v1 = np.random.uniform(-1, 1, (10000, 2))
v2 = np.random.uniform(-1, 1, (10000, 2))

pols = []
for idx, v in enumerate(v1):
    twov = [v, v2[idx]]
    unit_vectors = preprocessing.normalize(twov, norm='l2')
    mean_of_sum_of_unitv = np.mean(unit_vectors, axis=0)
    mag_meanv = np.linalg.norm(mean_of_sum_of_unitv)
    pols.append(mag_meanv)
#     v_normalized = preprocessing.normalize(twov, norm='l1')
#     v_mean = np.mean(v_normalized, axis=0)
#     pol = np.linalg.norm(v_mean, ord=np.inf)
#     pols.append(pol)

    
plt.figure(figsize=(20,10))
plt.hist(pols, bins=30)


In [ ]:
# https://www.sciencedirect.com/science/article/pii/S0960982217310138

v1 = np.random.uniform(-1, 1, (1000, 2))
v2 = np.random.uniform(-1, 1, (1000, 2))

pols = []

for idx, v in enumerate(v1):
    angle1 = np.arctan2(v[1], v[0])
    angle2 = np.arctan2(v2[idx][1], v2[idx][0])
    
    sm = (np.square(np.sin(angle1) + np.sin(angle2))) + (np.square(np.cos(angle1) + np.cos(angle2)))
    rt = np.sqrt(sm)
    pol = rt/2
    
    pols.append(pol)

plt.hist(pols)
    
    

In [ ]:
a = "test12test1234.jpg"
a[-8:-4]